In [3]:
import pandas as pd

df = pd.read_csv('spam.csv', encoding='latin-1')[['v1','v2']]
df.columns = ['label', 'message']
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())


print(df['label'].value_counts())
n_spam = df['label'].value_counts()['spam']
n_ham = df['label'].value_counts()['ham']
percent_spam = n_spam / (n_spam + n_ham) * 100
print(f"Percentage spam: {percent_spam:.1f}%")

# Display 3 spam and 3 ham
print("Spam examples:\n", df[df['label']=='spam']['message'].head(3).tolist())
print("Ham examples:\n", df[df['label']=='ham']['message'].head(3).tolist())


Shape: (5572, 2)
Columns: ['label', 'message']
label
ham     4825
spam     747
Name: count, dtype: int64
Percentage spam: 13.4%
Spam examples:
 ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv", 'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.']
Ham examples:
 ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', 'U dun say so early hor... U c already then say...']


In [ ]:
Dataset shape: 5572 × 2 (messages × [label, message]).​

Columns: 'label', 'message'.​

Count spam vs ham: About 747 spam, 4825 ham.​

% messages spam: ~13.4%

In [5]:
# Compute priors
P_spam = n_spam/(n_spam+n_ham)
P_ham = n_ham/(n_spam+n_ham)

# Split data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Manually count "free" in spam vs ham in train set
n_free_spam = train[(train['label']=='spam') & (train['message'].str.contains('free', case=False))].shape[0]
n_free_ham = train[(train['label']=='ham') & (train['message'].str.contains('free', case=False))].shape[0]
n_spam_train = train['label'].value_counts()['spam']
n_ham_train = train['label'].value_counts()['ham']

P_free_given_spam = n_free_spam / n_spam_train
P_free_given_ham = n_free_ham / n_ham_train

print(f"P('free'|spam): {P_free_given_spam:.2f}")
print(f"P('free'|ham): {P_free_given_ham:.2f}")


P('free'|spam): 0.26
P('free'|ham): 0.01


In [ ]:
Prior probabilities: P(spam) ≈ 0.134, P(ham) ≈ 0.866.​

P('free'|spam): Typically much higher (e.g., ≈0.25–0.35) ​.

P('free'|ham): Very low (e.g., ≈0.01–0.02) ​.

If 'free' in message: It is much more likely to be spam, given these probabilities.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix


vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train['message'])
X_test = vectorizer.transform(test['message'])

y_train = train['label'].map(lambda x: 1 if x=='spam' else 0)
y_test = test['label'].map(lambda x: 1 if x=='spam' else 0)


model = MultinomialNB()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


Accuracy: 0.9838565022421525
Confusion Matrix:
 [[961   5]
 [ 13 136]]


In [ ]:
Accuracy: ~98% (typical for this dataset).

Better at spam/ham: Ham is predicted more accurately, but most spam is correctly detected.

In [9]:

import numpy as np
words = np.array(vectorizer.get_feature_names_out())
spam_prob = model.feature_log_prob_[1]
ham_prob = model.feature_log_prob_[0]

# Find strongest spam indicators
spam_ratios = np.exp(spam_prob - ham_prob)
top_idx = np.argsort(-spam_ratios)[:5]
print("Top spam indicators:", words[top_idx])

# Classify custom message
custom = ["Free call now! Win money!"]
custom_vec = vectorizer.transform(custom)
print("Custom prediction:", "Spam" if model.predict(custom_vec)[0]==1 else "Ham")


Top spam indicators: ['claim' 'prize' '150p' 'tone' '18']
Custom prediction: Spam


In [ ]:
Strongest spam indicators: Words like 'free', 'win', 'call', 'now', 'prize'.​

Custom message class: Will be classified as spam

In [11]:
msg1 = ["Call me"]
msg2 = ["Free call"]

print("Prob spam (Call me):", model.predict_proba(vectorizer.transform(msg1))[0][1])
print("Prob spam (Free call):", model.predict_proba(vectorizer.transform(msg2))[0][1])


Prob spam (Call me): 0.05646626480042218
Prob spam (Free call): 0.8452324743520473


In [ ]:
Probabilities: Free call yields a much higher spam probability than Call me.

Word independence: Naive Bayes assumes each word contributes independently to the spam probability (even when they are not truly independent).

Why does it still work? Many spam indicators are highly predictive even when word independence is not fully accurate; strong features make up for the naive assumption.